In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from post_process.performance import *
from post_process.estimate import *
import os
import seaduck as sd

from load_data.anomaly import *
from load_data.bolus_mask import separate_e_ua
from post_process.read import simple_read_nc
from post_process.estimate import separate_lhs
import sys
int_arg = 0

import os 
for i in range(1,9):
    os.listdir(f'/sciserver/filedb0{i}-02/')

numba not imported


In [35]:
# those_slices = [122,153,184,214,245]
# those_slices = [245,275,306]
# the_slice = slice(those_slices[int_arg], those_slices[int_arg+1])

particle_path = '/sciserver/filedb08-01/ocean/wenrui_temp/particle_file/neutral2001/'
print(os.listdir(particle_path))
seed = 2001
map_path = particle_path  +'time_traj_'+'maps/'
table_path = particle_path+'time_traj_'+'table/'
# output_path = particle_path+'output/' 
zarr_path = particle_path+'nc/'
vec0 = xr.open_zarr('sxsysz_mean')
oce = sd.OceData(vec0)

wrong = np.load('bolus_bug.npy')
bolus_mask = tuple(wrong)

path0 = '/sciserver/filedb10-01/ocean/wenrui_temp/'
path1 = '/sciserver/filedb11-01/ocean/wenrui_temp/'
path2 = '/sciserver/filedb12-01/ocean/wenrui_temp/'
all_file = []
first_patch = []
for path in [path0,path1,path2]:
    all_file+=[path + i for i in os.listdir(path)]
    for var in os.listdir(path):
        if 'particle' in var:
            continue
        elif '.zip' in var:
            continue
        elif 'centerS' in var:
            continue
        elif 'uall' in var:
            continue
        elif 'table' in var:
            continue
        elif 'walls_normal' in var:
            continue
        elif 'E_ua_mean' in var:
            E_ua_mean_name = path+var
        elif 'tendS_0N-1' in var:
            tends_fl_name  = path+var
        else:
            first_patch.append(path+var)
print('I want to print')
ds = xr.open_mfdataset(first_patch,engine = 'zarr', parallel = True)
ds = ds.drop_vars('E_ua_mean')

new = xr.open_zarr(E_ua_mean_name)
ds = xr.merge([ds,new])

tend_ = xr.open_zarr(tends_fl_name)
ds.tendS[0] = tend_.tendS_first
ds.tendS[-1]= tend_.tendS_last

['nc', 'mean_traj_maps', 'nc_mean', 'mean_traj_table']
I want to print


In [3]:
ds['R'] = -ds['ubargradsprime']
ds['e_ua'] = ds.E_ua-ds.E_ua_mean
ds['e_ssh'] = (ds.E_ssh-ds.E_ssh_mean)
ds['E'] = (-ds["u'grads'_mean"])
ds['dif_h'] = (ds.dif_hConvS-ds.dif_hConvS_mean).transpose('time','Z','face','Y','X')
ds['dif_v'] = (ds.dif_vConvS-ds.dif_vConvS_mean).transpose('time','Z','face','Y','X')
ds['I'] = (ds.forcS-ds.forcS_mean)
ds['A'] = ds["uprimegradsbar"]
ds['U'] = ds.tendS_mean - ds.tendS
ds['F'] = (ds.pe_mean - ds.pe)
print('I want to print!')

I want to print!


In [4]:
rhs_list = ['e_ua','E','dif_h','dif_v','A','I','F']
lhs_list = ['e_ssh', 'U']
termlist = rhs_list+lhs_list
for var in termlist:
    if 'time' in ds[var].dims:
        ds[var] = ds[var].transpose('time','Z','face','Y','X')
    else:
        ds[var] = ds[var].transpose('Z','face','Y','X')

target = 's'
indlist = ['ix', 'iy', 'izl_lin', 'face']
notrequired = ['rzl_lin', 'ry', 'rx','rt']
vellist = [ 'u', 'v', 'w', 'du', 'dv', 'dw']
morelist = ['lon', 'lat', 'dep', 'vs']

varlist = termlist+indlist+morelist+vellist+[target]

In [5]:
vec = xr.open_zarr(path1+'walls_normal')
vec_mean = xr.open_zarr('sxsysz_mean')
vec = xr.merge([vec,vec_mean])
vec['sxprime'] = vec['sx'] - vec['osx_mean']
vec['syprime'] = vec['sy'] - vec['osy_mean']
vec['szprime'] = vec['sz'] - vec['osz_mean']

In [9]:
filedb_lst = []
for i in range(1,13):
    for j in range(1,4):
        filedb_lst.append(f'/sciserver/filedb{i:02}-0{j}/')
        os.listdir(f'/sciserver/filedb{i:02}-0{j}/')

In [10]:
# path_mid = 'ocean/wenrui_temp/particle_file/meanM/nc/'
# which_node = list(range(9))
# spread = len(which_node)
# zarr_path = filedb_lst[which_node[0]]+path_mid
# duration = len(os.listdir(zarr_path))
# dates = [i[-19:] for i in sorted(os.listdir(filedb_lst[which_node[0]]+path_mid),reverse = True)]
# all_files = []
# for node in which_node:
#     path = filedb_lst[node]+path_mid
#     all_files.append([path+i for i in sorted(os.listdir(path),reverse = True)])
# all_files = np.array(all_files)

# print('I wanna print!!')

In [11]:
import os

In [23]:
path_mid = 'ocean/wenrui_temp/particle_file/neutral2001/nc/'
which_node = list(range(19,28))
spread = len(which_node)
duration = len(os.listdir(filedb_lst[which_node[0]]+path_mid))
dates = [i[-19:] for i in sorted(os.listdir(filedb_lst[which_node[0]]+path_mid),reverse = True) if f'Seed{seed}' in i]
all_files = []
for node in which_node:
    path = filedb_lst[node]+path_mid
    # print(path)
    all_files.append([path+i for i in sorted(os.listdir(path),reverse = True) if f'Seed{seed}' in i])
all_files = np.array(all_files)

In [25]:
files = all_files[:,0]
datasets = [xr.open_zarr(files[i]) for i in range(len(files))]

In [26]:
Np = sum([len(dataset.shapes) for dataset in datasets])

In [27]:
Np

45336

In [28]:
# peek = xr.open_zarr(zarr_path+f'Seed{seed}_2005-05-31T00:00:00')
# Np = len(peek.shapes)

particle_slc = slice(None)
dates = sorted([i[8:] for i in os.listdir(filedb_lst[which_node[0]]+path_mid) if 'zarr' not in i],reverse = True)
date_identifier = dates[particle_slc]
ds['index_of_time'] = xr.DataArray(np.arange(len(ds.time)),dims  ='time')
first_index = int(ds['index_of_time'].sel(time = '2001-10-01').values)
to_isel = np.arange(first_index, first_index - len(date_identifier),-1)

/tmp/ipykernel_3468250/3882342855.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  first_index = int(ds['index_of_time'].sel(time = '2001-10-01').values)


In [29]:
hovmoller = xr.Dataset()
for var in rhs_list+['lhs', 'sf', 'sl', 'lon', 'lat', 'dep']:
    hovmoller[var] = xr.DataArray(np.zeros((len(date_identifier),Np)),dims = ('time','space'))
print('I want to print!!')

I want to print!!


In [30]:
hovmoller

<xarray.Dataset>
Dimensions:  (time: 1825, space: 45336)
Dimensions without coordinates: time, space
Data variables: (12/13)
    e_ua     (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    E        (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    dif_h    (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    dif_v    (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    A        (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    I        (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ...       ...
    lhs      (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    sf       (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    sl       (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    lon      (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    lat      (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    dep      (time, space) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [31]:
def cumu_map_zarr(neo, contr_dic, varname, last):
    value = contr_dic[varname]
    ind = tuple(np.array(i)[:-1] for i in [neo.iz-1, neo.face, neo.iy, neo.ix])
    array = np.zeros((50,13,90,90))
    np.add.at(array, ind, value)
    return array

def cumu_map_array(neo, value):
    ind = tuple(np.array(i)[:-1] for i in [neo.iz-1, neo.face, neo.iy, neo.ix])
    array = np.zeros((50,13,90,90))
    np.add.at(array, ind, value)
    return array

maps = {}
for var in rhs_list+['count']:
    maps[var] = np.zeros((50,13,90,90))

In [32]:
t1 = time.time()
for it,(dataset_date, particle_date) in enumerate(zip(to_isel,date_identifier)):
    my = ds.isel(time = dataset_date)
    vec_i = vec.isel(time = dataset_date)

    prefetch_scalar = get_prefetch_scalar(my, termlist)
    prefetch_scalar = separate_e_ua(prefetch_scalar, bolus_mask, e_ua_name = 'e_ua')
    
    prefetch_vec = get_prefetch_vec(vec_i)
    more = np.array(prefetch_vec['s'][0])
    print('dataset_date:', particle_date, time.time()-t1)
    # for jt, date in enumerate(date_identifier):

    files = all_files[:,it]
    assert np.array([(particle_date in name) for name in files]).all()
    datasets = [xr.open_zarr(files[i],consolidated =False) for i in range(len(files))]
    ds0 = datasets[0]
    neo = xr.Dataset()
    neo['shapes'] = xr.concat([ds.shapes for ds in datasets], dim = 'shapes')
    nprof = [len(ds.nprof) for ds in datasets]
    prefix = [0]+list(accumulate(nprof))
    neo['wrong_ind'] = xr.concat([datasets[i].wrong_ind+prefix[i] for i in range(len(datasets))], dim = 'wrong_ind')
        
    for var in ['face','frac','ind1','ind2','ix','iy','iz','tres','tt','vs','xx','yy','zz']:
        neo[var] = xr.concat([ds[var] for ds in datasets], dim = 'nprof')
    
    s1 = more[tuple(neo['ind1'].data)]
    s2 = more[tuple(neo['ind2'].data)]
    
    frac = np.array(neo.frac)
    s_wall = s1*frac + (1-frac)*s2
    first, last = first_last(np.array(neo.shapes))
    tres = np.array(neo.tres)
    
    step_dic = simple_read_nc(neo,termlist,prefetch_scalar)
    
    nstep = len(frac)-1
    deltas = np.nan_to_num(np.diff(s_wall))
    deltas[last[:-1]] = 0
    tres_used = -tres[1:]
    tres_used[last[:-1]] = 0 

    correction = separate_lhs(neo, step_dic, last, lhs_name = ['U', 'e_ssh'])
    rhs_contr = deltas - correction

    contr_dic = term_p_relaxed(rhs_contr, tres_used, step_dic, rhs_list, neo.wrong_ind)
    # contr_dic = term_indie(rhs_contr, tres_used, step_dic, rhs_list)
    contr_dic['lhs'] = correction

    for var in rhs_list+['lhs']:
        cumsum = np.cumsum(np.nan_to_num(contr_dic[var]))
        cumsum = np.insert(cumsum[last-1],0,0)
        hovmoller[var][it,:] = np.diff(cumsum)
        # assert np.isclose(np.nansum(hovmoller[var][it]),np.nansum(contr_dic[var]))
    hovmoller['sl'][it,:] = s_wall[last]
    hovmoller['sf'][it,:] = s_wall[first]
    hovmoller['lon'][it,:] = np.array(neo.xx[first])
    hovmoller['lat'][it,:] = np.array(neo.yy[first])
    hovmoller['dep'][it,:] = np.array(neo.zz[first])

    for var in rhs_list:
        maps[var] += cumu_map_zarr(neo, contr_dic, var, last)
    maps['count'] += cumu_map_array(neo, np.ones_like(tres_used))

dataset_date: _2001-09-30T00:00:00 2.951857089996338
dataset_date: _2001-09-29T00:00:00 10.865259408950806
dataset_date: _2001-09-28T00:00:00 17.605956554412842
dataset_date: _2001-09-27T00:00:00 23.947709560394287
dataset_date: _2001-09-26T00:00:00 30.036831855773926
dataset_date: _2001-09-25T00:00:00 36.28426146507263
dataset_date: _2001-09-24T00:00:00 43.659197092056274
dataset_date: _2001-09-23T00:00:00 51.5514862537384
dataset_date: _2001-09-22T00:00:00 59.07714128494263
dataset_date: _2001-09-21T00:00:00 66.07700753211975
dataset_date: _2001-09-20T00:00:00 74.64653754234314
dataset_date: _2001-09-19T00:00:00 82.34853959083557
dataset_date: _2001-09-18T00:00:00 89.0107741355896
dataset_date: _2001-09-17T00:00:00 95.14616775512695
dataset_date: _2001-09-16T00:00:00 101.44260621070862
dataset_date: _2001-09-15T00:00:00 107.88226747512817
dataset_date: _2001-09-14T00:00:00 116.27597570419312
dataset_date: _2001-09-13T00:00:00 124.00921320915222
dataset_date: _2001-09-12T00:00:00 130.

In [37]:
maps_ds = xr.Dataset()
for var in rhs_list+['count']:
    maps_ds[var] = xr.DataArray(maps[var],dims = ('Z','face','Y','X'))
maps_ds.to_zarr(map_path, mode = 'w')

In [38]:
hovmoller.to_zarr(table_path, mode = 'w')
print('success')

success


In [39]:
map_path

'/sciserver/filedb08-01/ocean/wenrui_temp/particle_file/neutral2001/time_traj_maps/'